# Counting DNA Nucleotides

In [ ]:
# by myself
a_cnt, c_cnt, g_cnt, t_cnt = 0,0,0,0
with open('../data/rosalind_dna.txt', 'r') as f:
    dna = f.read()
    for letter in dna:
        if letter == 'A':
            a_cnt+=1
        elif letter == 'C':
            c_cnt+=1
        elif letter == 'G':
            g_cnt+=1
        elif letter == 'T':
            t_cnt+=1
                
print(a_cnt, c_cnt, g_cnt, t_cnt)

# Transcribing DNA into RNA

In [ ]:
# by myself
rna = ''
with open('../data/rosalind_rna.txt', 'r') as f:
    dna = f.read()
    rna = dna.replace('T', 'U')

print(rna)

# Complementing a Strand of DNA

In [ ]:
# by myself
reverse_complement = ''
with open('../data/rosalind_revc.txt', 'r') as f:
    dna = f.read()
    reverse_complement = dna[::-1].translate(str.maketrans('ATGC', 'TACG'))
    
print(reverse_complement)

# Rabbits and Recurrence Relations

In [ ]:
# by myself
# 1: a pair of 2 children -> 1
# 2: a pair of 2 adults -> 1
# 3: a pair of 2 adults and k pair of 2 children -> k+1
# ...
# F_n: # of alive rabbits
# F_n = F_(n-1) + k * F_(n-2)

with open('../data/rosalind_fib.txt', 'r') as f:
    inp = f.read().split()
    n, k = int(inp[0]), int(inp[1])
    dp = [0 for _ in range(n)]
    dp[0] = 1
    dp[1] = 1
    for i in range(2, n):
        dp[i] = dp[i-1] + k * dp[i-2]
    print(dp[n-1])  

In [ ]:
# by others
def fib(n, k):
    a, b = 1, 1
    for i in range(2, n):
        a, b = b, k*a + b
    return b

# Computing GC Content

In [ ]:
# by myself
from Bio import SeqIO

with open('../data/rosalind_gc.txt', 'r') as f:
    max_gc = -1
    max_name = ''
    records = SeqIO.parse(f, 'fasta')
    
    for record in records:
        seq = str(record.seq)
        gorc = [1 if l in 'GC' else 0 for l in seq]
        gc = sum(gorc) / len(gorc) * 100
        if gc > max_gc:
            max_gc = gc
            max_name = record.id

    print(max_name)
    print(max_gc)

# Counting Point Mutations

In [ ]:
# by myself
with open('../data/rosalind_hamm.txt', 'r') as f:
    dna1 = f.readline()
    dna2 = f.readline()
    
    cnt = sum([1 if n1!=n2 else 0 for n1, n2 in zip(dna1, dna2)])
    print(cnt)

In [ ]:
# by others
from operator import ne

with open("../data/rosalind_hamm.txt", 'r') as infile:
    print(sum(map(ne, *infile.read().split())))

# Mendel's First Law

In [ ]:
with open('../data/rosalind_iprb.txt', 'r') as f:
    inp = list(map(lambda x: int(x), f.read().split()))
    [k,m,n] = inp[:3]
    total = k+m+n
    print((k*(total-1) + m*(k+(m-1)*0.75+n*0.5) + n*(k+m*0.5)) / (total * (total-1)))

# Translating RNA into Protein

In [ ]:
"""
UUU F      CUU L      AUU I      GUU V
UUC F      CUC L      AUC I      GUC V
UUA L      CUA L      AUA I      GUA V
UUG L      CUG L      AUG M      GUG V
UCU S      CCU P      ACU T      GCU A
UCC S      CCC P      ACC T      GCC A
UCA S      CCA P      ACA T      GCA A
UCG S      CCG P      ACG T      GCG A
UAU Y      CAU H      AAU N      GAU D
UAC Y      CAC H      AAC N      GAC D
UAA Stop   CAA Q      AAA K      GAA E
UAG Stop   CAG Q      AAG K      GAG E
UGU C      CGU R      AGU S      GGU G
UGC C      CGC R      AGC S      GGC G
UGA Stop   CGA R      AGA R      GGA G
UGG W      CGG R      AGG R      GGG G 
"""
# creating a dict of codon table
codon_table = {}
with open('../data/rna_codon_table.txt', 'r') as f:
    line = f.readline()
    while line:
        l = line.split()
        for i in range(len(l)//2):
            codon_table[l[i*2]] = l[i*2+1]
        line = f.readline()

with open('../data/rosalind_prot.txt', 'r') as f:
    ans = ""
    rnas = f.read()
    for i in range(len(rnas)//3):
        code = codon_table[rnas[3*i:3*(i+1)]]
        if code == "Stop": 
            break
        ans += code
    print(ans)

# Finding a Motif in DNA

In [ ]:
# implement BM algorithm
from collections import defaultdict

def BM(s, t, skip):
    i = 0
    n = len(s)
    m = len(t)
    while i <= n - m:
        j = m - 1
        while j >= 0 and s[i+j] == t[j]:
            j = j - 1
        if j == -1: return i
        i += max(skip[s[i+j]] + j - m + 1, 1)
    return -1

def skip(t):
    d = defaultdict(lambda: len(t))
    for i, l in enumerate(t):
        d[l] = len(t) - 1 - i
    return d
    
with open('../data/rosalind_subs.txt', 'r') as f:
    s = f.readline()[:-1]
    t = f.readline()[:-1]
    skip = skip(t)
    
    ans = ''
    i = 0
    while True:
        ind = BM(s[i:],t,skip)
        if ind < 0: break
        ans += str(i+ind+1) + ' '
        i += ind + 1
    print(ans[:-1])

# Consensus and Profile

profile matrix:  
長さnのDNA鎖の集合が与えられた時、(4,n)の行列  
成分(1,j)はj番目にAがどのくらい含まれているか。

consensus string:  
profile matrixの各j番において一番多かった物を取った鎖を示す文字列

In [ ]:
import numpy as np
from Bio import SeqIO

def str2mat(s):
    def char2vec(c):
        if c == 'A': return [1,0,0,0]
        elif c == 'C': return [0,1,0,0]
        elif c == 'G': return [0,0,1,0]
        elif c == 'T': return [0,0,0,1]
    return np.array([char2vec(c) for c in s])

def arg2char(a):
    if a == 0: return 'A'
    elif a == 1: return 'C'
    elif a == 2: return 'G'
    elif a == 3: return 'T'

with open('../data/rosalind_cons.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    mat = str2mat(str(records.__next__().seq))
    for record in records:
        mat += str2mat(str(record.seq))
    
    print(''.join([arg2char(a) for a in np.argmax(mat, axis=1)]))
    print('A:',' '.join([str(e) for e in mat[:,0]]))
    print('C:',' '.join([str(e) for e in mat[:,1]]))
    print('G:',' '.join([str(e) for e in mat[:,2]]))
    print('T:',' '.join([str(e) for e in mat[:,3]]))

# Mortal Fibonacci Rabbits

In [ ]:
'''
F_1: a pair of 2 children -> 1 pair
F_2: a pair of 2 adults -> 1 pair
F_3: a pair of 2 adults and a pair of 2 children -> 2 pair
...
F_k = F_k-1 + F_k-2

However, different from Rabbits and Recurrence Relations, rabbits live only for m month
Therefore, F_k = F_k-1 + F_k-2 - F_k-m
'''

with open('../data/rosalind_fibd.txt', 'r') as f:
    inp = f.read().split()
    n, m = int(inp[0]), int(inp[1])
    children = [0 for _ in range(n)]
    adults = [0 for _ in range(n)]
    children[0] = 1
    adults[0] = 0
    for i in range(1,n):
        if i >= m:
            adults[i] = children[i-1] + adults[i-1] - children[i-m]
        else:
            adults[i] = children[i-1] + adults[i-1]
        children[i] = adults[i-1]
    
    print(adults[n-1] + children[n-1])

In [ ]:
# by others
def fib(n,k=1):
    ages = [1] + [0]*(m-1)
    for i in range(n-1):
        ages = [sum(ages[1:])] + ages[:-1]
    return sum(ages)

n = 6
m = 3
fib(6,3)

# Overlap Graphs

In [ ]:
'''
each node is connected ((s,t) is edge) if 3 suffix of s and 3 prefix of t are equal
'''
from collections import defaultdict
from Bio import SeqIO
import itertools

with open('../data/rosalind_grph.txt', 'r') as f:
    k = 3
    suf_dic = defaultdict(list)
    pre_dic = defaultdict(list)
    records = SeqIO.parse(f, 'fasta')
    for record in records:
        id_part = record.id
        seq = str(record.seq)
        pre_dic[seq[:k]] = pre_dic[seq[:k]] + [id_part]
        suf_dic[seq[-k:]] = suf_dic[seq[-k:]] + [id_part]
        
    
    dna = ('A', 'C', 'G', 'T')
    ans = []
    for i in itertools.product(dna, repeat=k):
        key = ''.join(i)
        ans += itertools.product(suf_dic[key], pre_dic[key])
    
    print('\n'.join(set([a + ' ' + b for (a,b) in ans if a != b])))

# Calculating Expected Offspring

In [ ]:
import numpy as np
with open('../data/rosalind_iev.txt', 'r') as f:
    inp = np.array([int(n) for n in f.read().split()])
    e = np.array([2, 2, 2, 1.5, 1, 0])
    print(np.dot(inp, e))

# Finding a Shared Motif

In [ ]:
# not using but leave for future
def lcs(x,y):
    dp = [[0 for _ in range(len(y)+1)] for _ in range(len(x)+1)]
    longest, x_longest = 0, 0
    for i, e1 in enumerate(x):
        for j, e2 in enumerate(y):
            if e1==e2:
                dp[i+1][j+1] = dp[i][j] + 1
                if dp[i+1][j+1] > longest:
                    longest = dp[i+1][j+1]
                    x_longest = i+1

    return x[x_longest-longest:x_longest]

from Bio import SeqIO

with open('../data/rosalind_lcsm.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    seqs = [str(record.seq) for record in records]
    s = seqs[0]
    ans = ''
    for i in range(len(s)):
        for j in range(i+1, len(s)+1):
            sub = s[i:j]
            flag = True
            for k in range(1, len(seqs)):
                if sub not in seqs[k]:
                    flag = False
                    break
            if len(sub) > len(ans) and flag:
                ans = sub
                
    print(ans)

# Independent Alleles

In [ ]:
'''
k: k-th generation
N: N Aa Bb organisms

Each organism always mates with an organism having genotype Aa Bb
Each chileren have 2 children

=> 0.25^N x 0.75^(2^k-N) + ... + 0.25^(2^k) x 0.75^(0)
'''
from scipy.special import comb
with open('../data/rosalind_lia.txt', 'r') as f:
    inp = f.read().split()
    k, N = int(inp[0]), int(inp[1])
    ans = 0
    for i in range(N, 2**k+1):
        ans += comb(2**k, i, exact=True) * (0.25**i) * (0.75**((2**k)-i))
    print(ans)

In [ ]:
# by others
from scipy.stats import binom
1 - binom.cdf(n - 1, 2 ** k, 0.25)

# Finding a Protein Motif

In [ ]:
'''
pattern: N{P}[ST]{P}
'''

import regex as re  # in order to use overlapped flag, which is not implemented in standard re
import urllib.request
from Bio import SeqIO

with open('../data/rosalind_mprt.txt', 'r') as f:
    pattern = r'N[^P](S|T)[^P]'
    repatter = re.compile(pattern)
    ids = f.read().split()
    for id_ in ids:
        url = 'https://www.uniprot.org/uniprot/' + str(id_) + '.fasta'
        protein = ''.join(urllib.request.urlopen(url).read().decode('utf-8').split('\n')[1:-1])
        
        ans = ''
        for match in repatter.finditer(protein, overlapped=True):
            ans += str(match.start()+1) + ' '
        if ans != '':
            print(id_)
            print(ans[:-1])

# Inferring mRNA from Protein

In [ ]:
from collections import defaultdict
codon_cnt_table = defaultdict(int)
with open('../data/rna_codon_table.txt', 'r') as f:
    line = f.readline()
    while line:
        l = line.split()
        for i in range(len(l)//2):
            codon_cnt_table[l[i*2+1]] += 1
        line = f.readline()
        
with open('../data/rosalind_mrna.txt', 'r') as f:
    line = f.readline()  # there should be only one line
    ans = 3  # for stop codon
    for protein in line[:-1]:
        ans = (ans * codon_cnt_table[protein]) % 1000000
    print(ans)

# Open Reading Frames
open reading frame (ORF) is one which starts from the start codon and ends by stop codon, without any other stop codons in between.

In [ ]:
from Bio import SeqIO

# creating a dict of codon table
dna_codon_table = {}
with open('../data/dna_codon_table.txt', 'r') as f:
    line = f.readline()
    while line:
        l = line.split()
        for i in range(len(l)//2):
            dna_codon_table[l[i*2]] = l[i*2+1]
        line = f.readline()

with open('../data/rosalind_orf.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    seq = records.__next__().seq
    s_ = str(seq)
    s_comp = seq.reverse_complement()
    ans_set = set()
    start_codon = 'ATG'
    for s in [s_, s_comp]:
        for i in range(len(s)-3):
            if s[i:i+3] == start_codon:
                ans = 'M'
                k = i + 3
                while k + 3 <= len(s):
                    if dna_codon_table[s[k:k+3]] == 'Stop':
                        ans_set.add(ans)
                        break
                    ans += dna_codon_table[s[k:k+3]]
                    k += 3
    print('\n'.join(ans_set))

# Enumerating Gene Orders

In [ ]:
import itertools

with open('../data/rosalind_perm.txt', 'r') as f:
    inp = f.read().split()
    n = int(inp[0])
    
    perm = list(itertools.permutations(range(1,n+1)))
    print(len(perm))
    for e in perm:
        print(' '.join([str(a) for a in e]))

# Calculating Protein Mass

In [ ]:
monoisotopic_mass_table = {}
with open('../data/monoisotopic_mass_table.txt', 'r') as f:
    line = f.readline()
    while line:
        l = line.split()
        monoisotopic_mass_table[l[0]] = float(l[1])
        line = f.readline()

with open('../data/rosalind_prtm.txt', 'r') as f:
    inp = f.read()[:-1]
    print(sum([monoisotopic_mass_table[e] for e in inp]))

# Locating Restriction Sites

In [ ]:
from Bio import SeqIO

with open('../data/rosalind_revp.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    seq = records.__next__().seq
    seq_rev_comp = seq.reverse_complement()
    seq = str(seq)
    for i in range(2, len(seq)-1):
        k = 2
        length = 4
        position = i - 1
        while k <= 7 and i - k >= 0 and i + k <= len(seq):
            if seq[i-k:i+k] == seq_rev_comp[len(seq)-i-k:len(seq)-i+k]:
                print(position, length)
                length += 2
                position -= 1
            else:
                break
            k += 1

# RNA Splicing

In [ ]:
from Bio import SeqIO
import numpy as np
import re

# creating a dict of codon table
dna_codon_table = {}
with open('../data/dna_codon_table.txt', 'r') as f:
    line = f.readline()
    while line:
        l = line.split()
        for i in range(len(l)//2):
            dna_codon_table[l[i*2]] = l[i*2+1]
        line = f.readline()
        
with open('../data/rosalind_splc.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    seq = str(records.__next__().seq)
    intron_inds = []
    for record in records:
        sub_seq = str(record.seq)
        for match in re.finditer(sub_seq, seq):
            intron_inds.extend(list(range(match.start(), match.end())))
    seq_arr = np.array(list(seq))
    ind = np.ones(len(seq), dtype=bool)
    ind[sorted(intron_inds)] = False
    spliced_seq = seq_arr[ind]
    
    k = 0
    ans = ''
    while k+3 <= len(spliced_seq) and dna_codon_table[''.join(spliced_seq[k:k+3])] != 'Stop':
        ans += dna_codon_table[''.join(spliced_seq[k:k+3])]
        k += 3
    print(ans)

# Enumerating k-mers Lexicographically

In [ ]:
import itertools

with open('../data/rosalind_lexf.txt', 'r') as f:
    symbols = f.readline().split()
    n = int(f.readline())
    [print(''.join(e)) for e in list(itertools.product(symbols, repeat=n))]

# Longest Increasing Subsequence

In [ ]:
import math

def longest_inc_sub(s):
    P = [0 for _ in s]
    M = P + [0]
    L = 0
    for i in range(len(s)):
        # binary search
        lo = 1
        hi = L
        while lo <= hi:
            mid = math.ceil((lo+hi)/2)
            if s[M[mid]] < s[i]:
                lo = mid + 1
            else:
                hi = mid - 1
        newL = lo
        P[i] = M[newL-1]
        M[newL] = i
        if newL > L:
            L = newL
    S = [0 for _ in range(L)]
    k = M[L]
    for i in range(L)[::-1]:
        S[i] = s[k]
        k = P[k]
    return S

with open('../data/rosalind_lgis.txt', 'r') as f:
    n = int(f.readline())
    perms = [int(e) for e in f.readline().split()]
    print(' '.join([str(e) for e in longest_inc_sub(perms)]))
    print(' '.join([str(e) for e in longest_inc_sub(perms[::-1])[::-1]]))

# Genome Assembly as Shortest Superstring
A string is a superstring for a given collection of smaller strings if it contains each of the smaller strings as a substring.

In [ ]:
from Bio import SeqIO

def findOverlappingPair(s1, s2, s):
    max_ = -1
    len1 = len(s1)
    len2 = len(s2)
    # check suffix of s1
    for i in range(1, min(len1, len2)+1):
        if s2[:i] == s1[-i:] and max_ < i:
            max_ = i
            s = s1 + s2[i:]
    # check prefix of s1
    for i in range(1, min(len1, len2)+1):
        if s1[:i] == s2[-i:] and max_ < i:
            max_ = i
            s = s2 + s1[i:]
    return max_, s
        

with open('../data/rosalind_long.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    seqs = [str(record.seq) for record in records]
    while len(seqs) > 1:
        max_ = -1
        l, r = 0, 0
        s = ''
        resStr = ''
        for i in range(len(seqs)):
            for j in range(i+1, len(seqs)):
                res, s = findOverlappingPair(seqs[i], seqs[j], s)
                if max_ < res:
                    max_ = res
                    resStr = s
                    l, r = i, j
        if max_ == -1:
            seqs[0] += seqs[-1]
        else:
            seqs[l] = resStr
            seqs[r] = seqs[-1]
        seqs = seqs[:-1]
                    
    print(seqs[0])

# Perfect Matchings and RNA Secondary Structures

In [ ]:
from Bio import SeqIO
import math

with open('../data/rosalind_pmch.txt', 'r') as f:
    records = SeqIO.parse(f, 'fasta')
    seq = [str(record.seq) for record in records][0]
    nc = seq.count('C')
    na = seq.count('A')
    print(math.factorial(nc) * math.factorial(na))

# Partial Permutations
A partial permutation is an ordering of only k objects taken from a collection containing n objects (i.e., k≤n).  
ex: (5,7,2) is an example of partial permutation of three of the first eight positive integers.

In [ ]:
import math

with open('../data/rosalind_pper.txt', 'r') as f:
    inp = f.read().split()
    n, k = int(inp[0]), int(inp[1])
    print((math.factorial(n) // math.factorial(n-k)) % (10**6))